<a href="https://colab.research.google.com/github/Rehman047/Question-Answering-System--Pytorch-/blob/main/QA_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import torch

In [17]:
df=pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [18]:
def tokenize(text):
  text=text.lower()
  text=text.replace('?','')
  text=text.replace("'","")
  return text.split()

In [19]:
vocab={'UNV':0}
def build_vocab(row):
  text=tokenize(row['question'])+tokenize(row['answer'])
  for token in text:
    if token in vocab:
      continue
    vocab[token]=len(vocab)

In [20]:
df.apply(build_vocab,axis=1)


,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [21]:
len(vocab)

324

In [22]:
def text_to_indices(text,vocab):
  indices=[]
  for word in tokenize(text):
    if word not in vocab:
      word='UNV'
    indices.append(vocab[word])
  return indices

In [23]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [30]:
class Data(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
    return torch.tensor(text_to_indices(self.df.iloc[index]['question'],self.vocab)), torch.tensor(text_to_indices(self.df.iloc[index]['answer'],self.vocab))

In [31]:
data=Data(df,vocab)

In [32]:
data[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [33]:
loader=DataLoader(data,batch_size=1,shuffle=True)

In [35]:
for que,ans in loader:
  print(que)
  print(ans[0])
  print('--------')

tensor([[ 42, 250, 251, 118, 252, 253]])
tensor([254])
--------
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]])
tensor([199])
--------
tensor([[ 1,  2,  3, 69,  5, 53]])
tensor([260])
--------
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]])
tensor([113])
--------
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]])
tensor([185])
--------
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]])
tensor([61])
--------
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]])
tensor([244])
--------
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]])
tensor([52])
--------
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]])
tensor([23])
--------
tensor([[42, 43, 44, 45, 46, 47, 48]])
tensor([49])
--------
tensor([[10, 11, 12, 13, 14, 15]])
tensor([16])
--------
tensor([[ 42, 255,   2, 256,  83, 257, 258]])
tensor([259])
--------
tensor([[10, 29,  3, 30, 31]])
tensor([32])
--------
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]])
tensor([316])
--------
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]])